# Practical 1: Position and Orientation

We first import the required dependencies

In [1]:
# import libraries we need
import numpy as np

# Exercises

Complete each one of the following programming exercises and submit your solution by the deadline specified on Moodle

### Homogeneous transformation in 2D (3 points)

Given an angle $\theta$ and two displacements $d_x$ and $d_y$, code the corresponding homogeneous transformation matrix

In [2]:
def get_homogeneous_transform_2d(theta, dx, dy):
    
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    #TODO: construct the matrix tr -------------------------------
    tr = np.eye(3)
    tr[0, :] = np.array([cos_theta, -sin_theta, dx])
    tr[1, :] = np.array([sin_theta, cos_theta, dy])

    #ENDTODO -----------------------------------------------------
    
    return tr

We now test the implemented 2D homogeneous transformation with $\theta = \pi/2$, $dx = 5$ and $dy = 5$. Note that we are calling `np.round` because of the numerical error, such as `np.cos(np.pi/2) = 6.123233995736766e-17` instead of being equal to 0.

In [3]:
np.round(get_homogeneous_transform_2d(np.pi/2, 5, 5), 2)

array([[ 0., -1.,  5.],
       [ 1.,  0.,  5.],
       [ 0.,  0.,  1.]])

Expected output: 
```
array([[ 0., -1.,  5.],
       [ 1.,  0.,  5.],
       [ 0.,  0.,  1.]])
```

### Composition of homogeneous transformations  (4 points)

<!-- <img src="https://drive.google.com/uc?id=1dBXMNwNbwXsrnBRUwscGXHvh3OlvzPol"> -->
<img src="https://i.postimg.cc/v8fVnp0R/exercise-w1-images.png" width="500" height="500">

Suppose that the location of a robot (dashed frame) with respect to the world frame (solid frame) is given by the following sequence of transformations:
- A rotation by an angle $\alpha$ (in radians) about the current $z$-axis
- A translation of $b$ units along the current $y$-axis
- A rotation by an angle $\beta$ (in radians) about the current $x$-axis

The robot has a camera (dotted frame) whose location relative to the robot is given by:
- A rotation by an angle $\gamma$ (in radians) about the current $y$-axis
- A translation of $c$ units along the current $x$-axis


Given the coordinates of a points $p_c$ (light blue dot) in the camera frame, determine the coordinates of the same point in the robot's frame and the world frame. **Note that**: the image does not reflect the proper order of transformation, it is purely to visualize solid, dotted, and dashed line.

In [4]:
def get_point_in_robot_frame(point_c=np.array([0,0,1]), gamma=0, disp_c=0):
    
    # Represent point_c in homogeneous form
    homogeneous_point_c = np.hstack([point_c, 1])

    cos_gamma = np.cos(gamma)
    sin_gamma = np.sin(gamma)

    #TODO: Define rot_cam_rob to encode rotation defined by gamma----
    rot_cam_rob = np.eye(4)
    rot_cam_rob[0,0] = cos_gamma
    rot_cam_rob[2,2] = cos_gamma
    rot_cam_rob[0,2] = sin_gamma
    rot_cam_rob[2,0] = -sin_gamma
    # rot_cam_rob[1:3, 1:3] = np.array([[cos_gamma, -sin_gamma], [sin_gamma, cos_gamma]])
    #ENDTODO -------------------------------------------------------- 
    
    #TODO: Define trl_cam_rob to encode translation by disp_c--------
    trl_cam_rob = np.eye(4)
    trl_cam_rob[0, 3] = disp_c
    #ENDTODO --------------------------------------------------------
    
    #TODO: Define final_tr as the composed transformation from gamma and disp_c
    final_tr = np.eye(4)
    final_tr = np.matmul(rot_cam_rob, trl_cam_rob)
    #ENDTODO
    
    # Apply transformation to determine coordinates of point_c in robot frame
    point_r = np.matmul(final_tr, homogeneous_point_c)[:3]
    
    return point_r

def get_point_in_world_frame(point_c=np.array([0,0,1]),
                                alpha=0, beta=0, gamma=0, 
                                disp_c=0, disp_b=0):
    
    # Get point in robot frame
    point_r = get_point_in_robot_frame(point_c, gamma, disp_c)

    cos_alpha = np.cos(alpha)
    sin_alpha = np.sin(alpha)

    cos_beta = np.cos(beta)
    sin_beta = np.sin(beta)
    
    # Represent point_r in homogeneous form
    homogeneous_point_r = np.hstack([point_r, 1])
    
    #TODO: Define rot_z to encode rotation defined by alpha----------
    rot_z = np.eye(4)
    rot_z[0:2, 0:2] = np.array([[cos_alpha, -sin_alpha], [sin_alpha, cos_alpha]])
    #ENDTODO --------------------------------------------------------

    #TODO: Define trl_cam_world to encode translation by disp_b------
    trl_rob_world = np.eye(4)
    trl_rob_world[1, 3] = disp_b
    #ENDTODO --------------------------------------------------------
    
    #TODO: Define rot_x to encode rotation defined by beta-----------
    rot_x = np.eye(4)
    rot_x[1:3, 1:3] = np.array([[cos_beta, -sin_beta], [sin_beta, cos_beta]])

    #ENDTODO --------------------------------------------------------
    
    #TODO: compose effect of alpha, beta, disp_b into final_tr-------
    final_tr = np.eye(4)
    final_tr = np.matmul(np.matmul(rot_z, trl_rob_world), rot_x)
    #ENDTODO --------------------------------------------------------
    
    # Apply transformation to determine coordinates of point_r in world frame
    point_w = np.matmul(final_tr, homogeneous_point_r)[:3]
    
    return point_w

We now test the implemented 3D homogeneous transformation with a point $p_c$ in camera frame, with coodinate $x = 0, y = 1, z = 3$. Where would that point be in: 
- robot frame?
- world frame?

In [5]:
print(np.round(get_point_in_robot_frame(point_c=np.array([0, 1, 3]), gamma=0, disp_c=2)))
print(np.round(get_point_in_world_frame(point_c=np.array([0, 1, 3]), gamma=0, disp_c=2, alpha=0, beta=0, disp_b=-1)))

[2. 1. 3.]
[2. 0. 3.]


Expected output:
```
[2. 1. 3.]
[2. 0. 3.]
```

# Grading

Test cases for giving marks:
- `q1`: Assess the `get_homogeneous_transform_2d` function (3 points)
- `q2_robot`: Assess the `get_point_in_robot_frame` function (2 points)
- `q2_world`: Assess the `get_point_in_world_frame` function (2 points)

In [6]:
import otter
grader = otter.Notebook(tests_dir = "Practical01_Support/tests")
grader.check_all()

q1 results: All test cases passed!

q2_robot results: All test cases passed!

q2_world results: All test cases passed!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6afc435d-be2f-49be-9a92-b46145caf9a7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>